In [1]:
# Incorporating Dash
from dash import Dash,html,dcc, Input, Output
import dash_bootstrap_components as dbc
import pathlib
import tiledbvcf

In [2]:
# Incorporating Dash

from dash import Dash,html,dcc, Input, Output
import dash_bootstrap_components as dbc
import pathlib
import tiledbvcf


# Globally defined variables 


# Instantiating the app

app = Dash("__name__", external_stylesheets=[dbc.themes.SLATE] )

# App layout

app.layout = html.Div([
    html.H1("Hello Dash", style={'textAlign': 'center'}),
    dbc.Row(
        [
            dbc.Col('Column1', width= 6),
            dbc.Col('Column2', width=6)
        ]
    ),
    dbc.Row([
        dbc.Col('Column3', width=12, style={'textAlign': 'center'})
    ]),
    dcc.Dropdown(options=['blue','green'],value="blue", id='color_dropdown'),
    html.Div(id="color_output"),
    dcc.Input(placeholder="Enter the path of the directory containing vcf files", id= "path_to_files", type='text', value = "."),
    html.Div(id="folder_report"),
    dcc.Input(placeholder="Enter the name of the dataset to be created or default tiledbtest will be used.", id= "uri", type='text', value = "tiledbtest"),
    html.Div(id="dataset_creation_report")
]) 

@app.callback(Output("folder_report", "children"),
              Input("path_to_files", "value"),
             )
def file_report(file_path):
    if pathlib.Path(file_path).exists()==True:
        directory = pathlib.Path(file_path)
        files = list(directory.glob("*.vcf.gz"))
        number_of_files = len(files)
        return(f"The folder has {number_of_files} vcf files.")
    else:
        return("The filepath doesn;t exist. Please check it")

file_path="/Users/vibhor/Downloads/GIAB_vcfs/"

@app.callback(Output("dataset_creation_report", "children"),
              Input("uri", "value"))
def dataset_creation(uri):
    dataset_path =pathlib.Path( "./Data")/uri
    ds = tiledbvcf.Dataset(str(dataset_path),"w")
    directory = pathlib.Path(file_path)
    files = list(directory.glob("*.vcf.gz"))
    ds.create_dataset()
    print('HOLA')
    for file in files:
    	try:
    		ds.ingest_samples([str(file)])
    	except Exception as e:
            print(e)
    print("dataset created")
        
    
    


# Running the app
if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode = 'external', port=8051)

Dash app running on http://127.0.0.1:8051/
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
Cell In[2], line 58, in dataset_creation(uri='tiledbtest')
     56 directory = pathlib.Path(file_path)
     57 files = list(directory.glob("*.vcf.gz"))
---> 58 ds.create_dataset()
        ds = <tiledbvcf.dataset.Dataset object at 0x11f213d60>
     59 print('HOLA')
     60 for file in files:

File /opt/anaconda3/envs/tiledbtest/lib/python3.10/site-packages/tiledbvcf/dataset.py:691, in Dataset.create_dataset(
    self=<tiledbvcf.dataset.Dataset object>,
    extra_attrs=None,
    vcf_attrs=None,
    tile_capacity=10000,
    anchor_gap=1000,
    checksum_type='sha256',
    allow_duplicates=True,
    enable_allele_count=True,
    enable_variant_stats=True,
    compress_sample_dim=True,
    compression_level=4
)
    688     self.writer.set_compression_level(compression_level)
    690 # This call thro